<a href="https://colab.research.google.com/github/Jammond/ArtGAN/blob/master/TwitterAAE_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [48]:
import json
import pandas as pd
import re

In [10]:
data = open('/content/drive/MyDrive/Github/octo-rapper/twitteraae_all_aa').read()

# df = pd.read_csv('/content/drive/MyDrive/Github/octo-rapper/twitteraae_all_aa', sep='\t')

In [7]:
data[:500]

'293846693215096832\t"Tue Jan 22 22:24:45 +0000 2013"\t"1028920752"\t[-80.01040975, 32.80108357]\t"450190027021"\t"Click Clack Motha Fucka I ain\'t tryin to hear Nothin!"\t0.894545454545\t0.0163636363636\t0.0\t0.0890909090909\n288532237186646016\t"Tue Jan 08 06:27:00 +0000 2013"\t"1040150983"\t[-87.322161, 41.6006888]\t"180890106001"\t"Man imissed a called from my bae hella mad -_- @callme_KRAZII"\t0.942\t0.058\t0.0\t0.0\n289899995472224257\t"Sat Jan 12 01:01:59 +0000 2013"\t"1068611971"\t[-78.85113963, 42.909513]\t"3602'

In [13]:
data_split = [line for line in data.split('\n')]


In [15]:
data_split[:5]

['293846693215096832\t"Tue Jan 22 22:24:45 +0000 2013"\t"1028920752"\t[-80.01040975, 32.80108357]\t"450190027021"\t"Click Clack Motha Fucka I ain\'t tryin to hear Nothin!"\t0.894545454545\t0.0163636363636\t0.0\t0.0890909090909',
 '288532237186646016\t"Tue Jan 08 06:27:00 +0000 2013"\t"1040150983"\t[-87.322161, 41.6006888]\t"180890106001"\t"Man imissed a called from my bae hella mad -_- @callme_KRAZII"\t0.942\t0.058\t0.0\t0.0',
 '289899995472224257\t"Sat Jan 12 01:01:59 +0000 2013"\t"1068611971"\t[-78.85113963, 42.909513]\t"360290033021"\t"@devontekthomas OMG I keep sayin boo wen I mean boy"\t0.951111111111\t0.0311111111111\t0.0\t0.0177777777778',
 '289901379869351936\t"Sat Jan 12 01:07:29 +0000 2013"\t"1068611971"\t[-78.8510475, 42.90955088]\t"360290033021"\t"@devontekthomas I did not mean to say dat"\t0.957142857143\t0.0342857142857\t0.0\t0.00857142857143',
 '289902453367910401\t"Sat Jan 12 01:11:45 +0000 2013"\t"1068611971"\t[-78.85107559, 42.90955579]\t"360290033021"\t"@devontekthom

In [26]:
df = pd.DataFrame([x.split('\t') for x in data_split])

In [19]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,293846693215096832,"""Tue Jan 22 22:24:45 +0000 2013""","""1028920752""","[-80.01040975, 32.80108357]","""450190027021""","""Click Clack Motha Fucka I ain't tryin to hear...",0.894545454545,0.0163636363636,0.0,0.0890909090909
1,288532237186646016,"""Tue Jan 08 06:27:00 +0000 2013""","""1040150983""","[-87.322161, 41.6006888]","""180890106001""","""Man imissed a called from my bae hella mad -_...",0.942,0.058,0.0,0.0
2,289899995472224257,"""Sat Jan 12 01:01:59 +0000 2013""","""1068611971""","[-78.85113963, 42.909513]","""360290033021""","""@devontekthomas OMG I keep sayin boo wen I me...",0.951111111111,0.0311111111111,0.0,0.0177777777778
3,289901379869351936,"""Sat Jan 12 01:07:29 +0000 2013""","""1068611971""","[-78.8510475, 42.90955088]","""360290033021""","""@devontekthomas I did not mean to say dat""",0.957142857143,0.0342857142857,0.0,0.00857142857143
4,289902453367910401,"""Sat Jan 12 01:11:45 +0000 2013""","""1068611971""","[-78.85107559, 42.90955579]","""360290033021""","""@devontekthomas awww u do too""",0.975,0.01,0.0,0.015


In [29]:
df = df.rename(columns={0: 'tweet_ids', 1: 'timestamps', 2:'user_ids', 3:'lat_long', 4:'blockgroup', 5:'text', 6:'African-American', 7:'Hispanic', 8:'Asian', 9:'White' })

In [30]:
df.head()

,tweet_ids,timestamps,user_ids,lat_long,blockgroup,text,African-American,Hispanic,Asian,White
0,293846693215096832,"""Tue Jan 22 22:24:45 +0000 2013""","""1028920752""","[-80.01040975, 32.80108357]","""450190027021""","""Click Clack Motha Fucka I ain't tryin to hear...",0.894545454545,0.0163636363636,0.0,0.0890909090909
1,288532237186646016,"""Tue Jan 08 06:27:00 +0000 2013""","""1040150983""","[-87.322161, 41.6006888]","""180890106001""","""Man imissed a called from my bae hella mad -_...",0.942,0.058,0.0,0.0
2,289899995472224257,"""Sat Jan 12 01:01:59 +0000 2013""","""1068611971""","[-78.85113963, 42.909513]","""360290033021""","""@devontekthomas OMG I keep sayin boo wen I me...",0.951111111111,0.0311111111111,0.0,0.0177777777778
3,289901379869351936,"""Sat Jan 12 01:07:29 +0000 2013""","""1068611971""","[-78.8510475, 42.90955088]","""360290033021""","""@devontekthomas I did not mean to say dat""",0.957142857143,0.0342857142857,0.0,0.00857142857143
4,289902453367910401,"""Sat Jan 12 01:11:45 +0000 2013""","""1068611971""","[-78.85107559, 42.90955579]","""360290033021""","""@devontekthomas awww u do too""",0.975,0.01,0.0,0.015


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149900 entries, 0 to 1149899
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   tweet_ids         1149900 non-null  object
 1   timestamps        1149899 non-null  object
 2   user_ids          1149899 non-null  object
 3   lat_long          1149899 non-null  object
 4   blockgroup        1149899 non-null  object
 5   text              1149899 non-null  object
 6   African-American  1149899 non-null  object
 7   Hispanic          1149899 non-null  object
 8   Asian             1149899 non-null  object
 9   White             1149899 non-null  object
dtypes: object(10)
memory usage: 87.7+ MB


In [32]:
trim_col = df[['text', 'African-American']]

In [33]:
trim_col.head()

,text,African-American
0,"""Click Clack Motha Fucka I ain't tryin to hear...",0.894545454545
1,"""Man imissed a called from my bae hella mad -_...",0.942
2,"""@devontekthomas OMG I keep sayin boo wen I me...",0.951111111111
3,"""@devontekthomas I did not mean to say dat""",0.957142857143
4,"""@devontekthomas awww u do too""",0.975


In [35]:
trim_col.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [76]:
trim_col["aa"] = trim_col["African-American"].astype(str).astype(float)


# stripped_string = a_string.strip('"')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [78]:
trim_col["text"] = trim_col["text"].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
trim_col["text"] = trim_col["text"].strip('"')

AttributeError: ignored

In [68]:
trim_col.head()

,text,African-American,aa
0,"""Click Clack Motha Fucka I ain't tryin to hear...",0.894545454545,0.894545
1,"""Man imissed a called from my bae hella mad -_...",0.942,0.942000
2,"""@devontekthomas OMG I keep sayin boo wen I me...",0.951111111111,0.951111
3,"""@devontekthomas I did not mean to say dat""",0.957142857143,0.957143
4,"""@devontekthomas awww u do too""",0.975,0.975000


In [69]:
trim_final = trim_col.drop(['African-American'], axis=1)
trim_final.head()

,text,aa
0,"""Click Clack Motha Fucka I ain't tryin to hear...",0.894545
1,"""Man imissed a called from my bae hella mad -_...",0.942000
2,"""@devontekthomas OMG I keep sayin boo wen I me...",0.951111
3,"""@devontekthomas I did not mean to say dat""",0.957143
4,"""@devontekthomas awww u do too""",0.975000


In [70]:
trim_final = trim_final[trim_final['aa'] >= .5]

In [71]:
trim_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1143421 entries, 0 to 1149898
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   text    1143421 non-null  object 
 1   aa      1143421 non-null  float64
dtypes: float64(1), object(1)
memory usage: 26.2+ MB


In [72]:
trim_final.head()

,text,aa
0,"""Click Clack Motha Fucka I ain't tryin to hear...",0.894545
1,"""Man imissed a called from my bae hella mad -_...",0.942000
2,"""@devontekthomas OMG I keep sayin boo wen I me...",0.951111
3,"""@devontekthomas I did not mean to say dat""",0.957143
4,"""@devontekthomas awww u do too""",0.975000


In [109]:
tweets = trim_final['text'].str.strip('"').drop_duplicates()
tweets[:5]

0    Click Clack Motha Fucka I ain't tryin to hear ...
1    Man imissed a called from my bae hella mad -_-...
2    @devontekthomas OMG I keep sayin boo wen I mea...
3            @devontekthomas I did not mean to say dat
4                        @devontekthomas awww u do too
Name: text, dtype: object

In [110]:
tweets = tweets.tolist()
tweets[:10]

["Click Clack Motha Fucka I ain't tryin to hear Nothin!",
 'Man imissed a called from my bae hella mad -_- @callme_KRAZII',
 '@devontekthomas OMG I keep sayin boo wen I mean boy',
 '@devontekthomas I did not mean to say dat',
 '@devontekthomas awww u do too',
 '@devontekthomas wat r u doin boy',
 '@ZariaDove I know dats rite',
 "I'm trying to get tweeter...",
 'Wth......Kenya......lmbo...........next time pic a different event to prove your point............#RHOA',
 'Twrkndathunibun']

In [111]:
tweets  = [x for x in tweets if not x.startswith('RT')]

tweets = [i for i in tweets if not ('http://' in i or 'https://' in i)]

In [112]:
tweets[:5]

["Click Clack Motha Fucka I ain't tryin to hear Nothin!",
 'Man imissed a called from my bae hella mad -_- @callme_KRAZII',
 '@devontekthomas OMG I keep sayin boo wen I mean boy',
 '@devontekthomas I did not mean to say dat',
 '@devontekthomas awww u do too']

In [113]:
tweets = [re.sub(r"(?:\@|https?\://)\S+", "", line.lower()) for line in tweets]
tweets = [re.sub(" \d+", "", line) for line in tweets]

tweets[:50]

["click clack motha fucka i ain't tryin to hear nothin!",
 'man imissed a called from my bae hella mad -_- ',
 ' omg i keep sayin boo wen i mean boy',
 ' i did not mean to say dat',
 ' awww u do too',
 ' wat r u doin boy',
 ' i know dats rite',
 "i'm trying to get tweeter...",
 'wth......kenya......lmbo...........next time pic a different event to prove your point............#rhoa',
 'twrkndathunibun',
 'mm dis is unusual my phone dry af right',
 ' lol null jus dont wanna have ya hot and bothered',
 ' seem like it man',
 'feel good outside tonight',
 ' lol nall fool jus a guess',
 '\\" .* ready to get these tats b4 spring break .!\\"1000',
 'if u push da right buttons, she gone dance without da bands lol',
 "datum dryer broke so i'm at dis dry cleaner",
 'full af',
 'cooling in da hood',
 ' mm really',
 ' so but i got dreads tho ;)',
 ' wat u sorry for fool',
 ' you gone makee come looking for yo lil ass',
 'off work early',
 ' i see',
 ' i hear ya',
 ' and u still lil smh',
 'finna fu

In [108]:
# tweets = [re.sub("[^a-zA-Z0-9]+ ", "", line) for line in tweets]
# tweets = [''.join(e for e in string if e.isalnum()) for string in tweets]
tweets[:50]
# tweets = [''.join(e for e in string if e.isalnum()) for string in l_in]

['clickclackmothafuckaiainttryintohearnothin',
 'manimissedacalledfrommybaehellamad',
 'omgikeepsayinboowenimeanboy',
 'ididnotmeantosaydat',
 'awwwudotoo',
 'watrudoinboy',
 'iknowdatsrite',
 'imtryingtogettweeter',
 'wthkenyalmbonexttimepicadifferenteventtoproveyourpointrhoa',
 'twrkndathunibun',
 'mmdisisunusualmyphonedryafright',
 'lolnulljusdontwannahaveyahotandbothered',
 'seemlikeitman',
 'feelgoodoutsidetonight',
 'lolnallfooljusaguess',
 'readytogetthesetatsb4springbreak1000',
 'ifupushdarightbuttonsshegonedancewithoutdabandslol',
 'datumdryerbrokesoimatdisdrycleaner',
 'fullaf',
 'coolingindahood',
 'mmreally',
 'sobutigotdreadstho',
 'watusorryforfool',
 'yougonemakeecomelookingforyolilass',
 'offworkearly',
 'isee',
 'ihearya',
 'andustilllilsmh',
 'finnafuckwithdisweightsforaminbbl',
 'uwentoutlastnight',
 'noworries',
 'lolikrbutimjpfool',
 'no',
 'itshardtofindagoodwomanwhocancooknowadays',
 'lovingyoavima',
 'trappinaintdeadtheseniggasjustscared1000',
 'damnrightdontmak

- Tweet text is converted to lowercase. -- done
- All mentions are removed. This includes the mentions --done
appearing in a regular tweet and the user handles at the
beginning of a retweet, e.g. ``RT: @realTrump’’.

- Dates and years, such as ``2017’’, are converted to two
symbols representing date and year, respectively.
- All ratios, such as ``3/23’’, are replaced by a special
symbol.
- Integers and decimals are normalized to two special
symbols.
- All special characters, except hashtags symbol #, are
removed.